# Cluster some Data
## do clustering using Keras / Tensorflow / SciKit
### this example uses the band-limited RMS of some seismometers
* http://learningtensorflow.com/lesson6/
* https://codesachin.wordpress.com/2015/11/14/k-means-clustering-with-tensorflow/
* http://napitupulu-jon.appspot.com/posts/kmeans-ud120.html
* https://www.datascience.com/blog/introduction-to-k-means-clustering-algorithm-learn-data-science-tutorials
* http://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html

In [ ]:
%matplotlib inline

from __future__ import division
import matplotlib.cm as cm
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np
#import os
#import scipy.constants as scc
from scipy.io import loadmat
#import scipy.signal as sig
#import sys
from timeit import default_timer as timer

#import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense, Activation
from sklearn.cluster import KMeans
import hdbscan    # can be installed using pip or directly from GitHub

# List of non-awful colors
cList = [
         (0.1, 0.1, 1.0, 0.9),
         (1.0, 0.1, 0.1, 0.9),
         (0, 0.7, 0, 0.9),
         (1.0, 0, 0.9, 0.9),
         (0.8, 0.8, 0, 0.9),
         (0, 0.6, 0.9, 0.9),
         (1, 0.5, 0, 0.9),
         (0.5, 0.5, 0.5, 0.9),
         (0.4, 0, 0.5, 0.9),
         (0, 0, 0, 0.9),
         (0.5, 0.3, 0, 0.9),
         (0, 0.3, 0, 0.9),
        ]

# Now alter my matplotlib parameters 
rcParams.update({'axes.color_cycle': cList,  # this is depreceated; use prop_cycle
                    'axes.grid': True,
                     'font.family': 'serif',
                     'font.size': 8,
                     #'font.serif': 'Palatino Linotype',
                     'grid.color': 'grey',
                     'grid.linestyle': '-',
                     'grid.alpha': 0.5,
                     'grid.linewidth': 1,
                     'legend.borderpad': 0.2,
                     'legend.fancybox': True,
                     'legend.fontsize': 8,
                     'legend.framealpha': 0.7,
                     'legend.handletextpad': 0.1,
                     'legend.labelspacing': 0.2,
                     'legend.loc': 'best',
                     'lines.linewidth': 1.5,
                     'savefig.bbox': 'tight',
                     'savefig.pad_inches': 0.02,
                     'savefig.dpi': 200,
                     'text.usetex': False,
                     'text.latex.preamble': r'\usepackage{txfonts}',
                     'figure.figsize': (7,4),
                     })

ifo='H1'

In [ ]:
data = loadmat('Data/' + ifo + '_SeismicBLRMS.mat')
blrms = np.transpose(data['data'])
#channels = data['chans']
npts, nchans = blrms.shape
print(str(nchans) + " channels of minute trend")
nbands = 6
tt = np.arange(start=0, step=60, stop = npts*60)
tdays = tt / 60 / 60 / 24

### Plot the BLRMS minute trend of the seismic data

In [ ]:
plt.figure(figsize=(10,4))


chans = data['chans']
# plot the BLRMS for 1 sensor
for zz in range(nbands):
    chan = chans[zz]
    chan = chan.replace(ifo + ':ISI-GND_STS_','')
    chan = chan.replace('.mean, m-trend','')
    plt.semilogy(tdays, blrms[:,zz], alpha=0.75,
        c = cm.spectral(int(256*(zz/nbands))), label=r'$\mathrm{%s}$' % chan.replace('_','\_'))


plt.ylim([9, 2000])
plt.xlim([0,30])
plt.xlabel('Time [days]')
plt.legend(loc='best')
plt.show()

In [ ]:
random_state = 137
tic = timer()
n_clusters = 10
# k-means clustering
k_pred = KMeans(n_clusters=n_clusters, random_state=random_state).fit_predict(blrms)
toc = timer()

# hdbscan clustering
# http://hdbscan.readthedocs.io/en/latest/basic_hdbscan.html
nsensors = nchans/nbands
t_stride = 10          # time chunk in minutes
min_clust_size = t_stride
hclust = hdbscan.HDBSCAN(min_cluster_size=min_clust_size)
h_pred = hclust.fit_predict(blrms)
print "# of clusters = " + str(hclust.labels_.max())

print(str(round(toc     - tic, 1)) + " seconds for K-Means...")
print(str(round(timer() - toc, 1)) + " seconds for H DB Scan...")

### Plot the 6 bands of one sensor with color indicating cluster

In [ ]:
tdays = tt / 60 / 60 / 24
#plt.figure(figsize=(12, 6))
fig, ax = plt.subplots(nrows=6, ncols=1)

k=0
for row in ax:
    z = blrms[:,k]
    ii = np.where(z > 0)[0]
    z  = z[ii]
    row.scatter(tdays[ii], z, c=k_pred[ii], alpha=0.5, s=1, cmap=cm.spectral)
    row.set_yscale('log')
    row.set_yticks(np.logspace(0,4,5))
    row.set_ylim(np.median(z)/3, z.max()*1.05)
    #row.set_ylim(10, 3000)

    #plt.xlabel('Time [days]')
    #row.set_ylabel('Velocity [microns/sec]')
    row.set_xticks([0,7,14,21,28])
    row.set_xlim([0,30])
    if k < 5:
        row.set_xticklabels('')
    k += 1

row.set_xlabel('Time [days]')

fig.tight_layout(pad=0.1, w_pad=0.1, h_pad=0.1) # https://matplotlib.org/users/tight_layout_guide.html
plt.show()

In [ ]:
x = data['data']

In [ ]:
list(data.keys())

In [ ]:
data['chans']

In [ ]:
blrms[:,k].max()

In [ ]:
ii = np.where(blrms[:,k] > 0)
ii[0].shape

In [ ]:
h_pred